In [ ]:
 import org.apache.spark.mllib.feature.{StandardScaler, StandardScalerModel}
 import org.apache.spark.mllib.linalg.Vectors
 import org.apache.spark.mllib.util.MLUtils
 import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}
 import org.apache.spark.mllib.feature.Normalizer
 import scala.collection.mutable.HashMap

 val data = sc.textFile("/home/stavros/Downloads/wineQualityReds.csv")
 val header = data.first()
 val parsedData = data.filter(row => row != header).map(s => Vectors.dense(s.split(',')
 .drop(1).map(_.toDouble))).cache()
 //parsedData.saveAsTextFile("vectorData.txt")
 //val vectorData = MLUtils.loadVectors(sc, "vectorData.txt")
 val normalizer = new Normalizer(p = Double.PositiveInfinity)
 val normalized_data = parsedData.map(x =>  normalizer.transform(x))

 val numIterations = 50
 val errors = new HashMap[Int,Double]()
 for (numClusters <- 2 to 20) {     
 val clusters = KMeans.train(normalized_data, numClusters, numIterations)
 val WSSSE = clusters.computeCost(normalized_data)
  errors.put(numClusters, WSSSE)
}


import org.apache.spark.mllib.feature.{StandardScaler, StandardScalerModel}
import org.apache.spark.mllib.linalg.Vectors
import org.apache.spark.mllib.util.MLUtils
import org.apache.spark.mllib.clustering.{KMeans, KMeansModel}
import org.apache.spark.mllib.feature.Normalizer
import scala.collection.mutable.HashMap
data: org.apache.spark.rdd.RDD[String] = /home/stavros/Downloads/wineQualityReds.csv MapPartitionsRDD[2369] at textFile at <console>:100
header: String = "","fixed.acidity","volatile.acidity","citric.acid","residual.sugar","chlorides","free.sulfur.dioxide","total.sulfur.dioxide","density","pH","sulphates","alcohol","quality"
parsedData: org.apache.spark.rdd.RDD[org.apache.spark.mllib.linalg.Vector] = MapPartitionsRDD[2371] at map at <console>:102
normalizer: org.apache.spark.m...

In [ ]:
errors.toArray

res57: Array[(Int, Double)] = Array((17,21.150203348739186), (8,34.57589002694767), (11,28.585919426029495), (20,19.16095105448462), (2,107.13934470152421), (5,48.3775722945334), (14,24.74444258956602), (4,56.78355364701449), (13,25.532661968280756), (16,22.26203354181584), (7,39.86989170189125), (10,31.396331079109558), (19,20.168949221386264), (18,21.22341634145377), (9,31.85113634209725), (3,75.49565647455694), (12,26.293823911524647), (15,23.148475555201205), (6,42.411413514254185))


entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
val clusters = KMeans.train(normalized_data, 2, numIterations)


clusters: org.apache.spark.mllib.clustering.KMeansModel = org.apache.spark.mllib.clustering.KMeansModel@243f80e1


In [ ]:
clusters.clusterCenters

res60: Array[org.apache.spark.mllib.linalg.Vector] = Array([0.5788327370461465,0.0338725581836942,0.020011301098778453,0.1666257715575919,0.0056309725158454,0.40349633343290753,0.9675213942134802,0.06601698929968569,0.2180910512034508,0.04238828475464813,0.714994401855638,0.38312153624625206], [0.18039444291409718,0.011903512685698184,0.00538682352254432,0.052890252892119106,0.001927570269812699,0.3694765387729449,1.0,0.022183009281431215,0.0741005521280797,0.014625924011909038,0.22995980340258093,0.12500312447932746])


entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
import org.apache.spark.mllib.feature.PCA
import org.apache.spark.mllib.regression.LabeledPoint

val datapoints = normalized_data.map{x=> new LabeledPoint(clusters.predict(x),x)} 
val pca = new PCA(2).fit(datapoints.map(_.features))
val projected = datapoints.map(p => p.copy(features = pca.transform(p.features)))


import org.apache.spark.mllib.feature.PCA
import org.apache.spark.mllib.regression.LabeledPoint
datapoints: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[3696] at map at <console>:105
pca: org.apache.spark.mllib.feature.PCAModel = org.apache.spark.mllib.feature.PCAModel@4447283c
projected: org.apache.spark.rdd.RDD[org.apache.spark.mllib.regression.LabeledPoint] = MapPartitionsRDD[3700] at map at <console>:107


In [ ]:
projected.collect().toArray.map(_.features.toArray).map { case Array(f1,f2) => (f1,f2) }

res63: Array[(Double, Double)] = Array((-0.3702973306866152,-0.33771637980173497), (-0.18712918469358497,-0.4023322671905787), (-0.22830404336207674,-0.30380330949811646), (-0.23880490950702832,-0.30385624143161144), (-0.3702973306866152,-0.33771637980173497), (-0.3103399519170776,-0.34436979708419047), (-0.19848360865860548,-0.28268066530572394), (-0.6977010617039199,-0.7043676334409302), (-0.7960129637347033,-0.4784408899780102), (-0.10492361385769952,-0.2037650030505796), (-0.1630245984212077,-0.2638485404888952), (-0.10492361385769952,-0.2037650030505796), (-0.18529130939240113,-0.30476015011481344), (-0.4362691590601039,-0.31712672795491115), (-0.07537772221534685,-0.3966274942156468), (-0.07221040405907608,-0.3829401962619964), (-0.12518177227143848,-0.3750539093048038), (-0.21478...

entries total 
 
 
 
 
 
 
 
 
 entries total

In [ ]:
clusters.toPMML()

res65: String =
"<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<PMML version="4.2" xmlns="http://www.dmg.org/PMML-4_2">
    <Header description="k-means clustering">
        <Application name="Apache Spark MLlib" version="2.2.0"/>
        <Timestamp>2017-10-31T13:18:51</Timestamp>
    </Header>
    <DataDictionary numberOfFields="12">
        <DataField name="field_0" optype="continuous" dataType="double"/>
        <DataField name="field_1" optype="continuous" dataType="double"/>
        <DataField name="field_2" optype="continuous" dataType="double"/>
        <DataField name="field_3" optype="continuous" dataType="double"/>
        <DataField name="field_4" optype="continuous" dataType="double"/>
        <DataField name="field_5" optype="continuous" dataType="double"/>
      ...

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<PMML version="4.2" xmlns="http://www.dmg.org/PMML-4_2">
 <Header description="k-means clustering">
 <Application name="Apache Spark MLlib" version="2.2.0"/>
 <Timestamp>2017-10-31T13:18:51</Timestamp>
 </Header>
 <DataDictionary numberOfFields="12">
 <DataField name="field_0" optype="continuous" dataType="double"/>
 <DataField name="field_1" optype="continuous" dataType="double"/>
 <DataField name="field_2" optype="continuous" dataType="double"/>
 <DataField name="field_3" optype="continuous" dataType="double"/>
 <DataField name="field_4" optype="continuous" dataType="double"/>
 <DataField name="field_5" optype="continuous" dataType="double"/>
 <DataField name="field_6" optype="continuous" dataType="double"/>
 <DataField name="field_7" optype="continuous" dataType="double"/>
 <DataField name="field_8" optype="continuous" dataType="double"/>
 <DataField name="field_9" optype="continuous" dataType="double"/>
 <DataField name="field_10" optype="continuous" dataType="double"/>
 <DataField name="field_11" optype="continuous" dataType="double"/>
 </DataDictionary>
 <ClusteringModel modelName="k-means" functionName="clustering" modelClass="centerBased" numberOfClusters="2">
 <MiningSchema>
 <MiningField name="field_0" usageType="active"/>
 <MiningField name="field_1" usageType="active"/>
 <MiningField name="field_2" usageType="active"/>
 <MiningField name="field_3" usageType="active"/>
 <MiningField name="field_4" usageType="active"/>
 <MiningField name="field_5" usageType="active"/>
 <MiningField name="field_6" usageType="active"/>
 <MiningField name="field_7" usageType="active"/>
 <MiningField name="field_8" usageType="active"/>
 <MiningField name="field_9" usageType="active"/>
 <MiningField name="field_10" usageType="active"/>
 <MiningField name="field_11" usageType="active"/>
 </MiningSchema>
 <ComparisonMeasure kind="distance">
 <squaredEuclidean/>
 </ComparisonMeasure>
 <ClusteringField field="field_0" compareFunction="absDiff"/>
 <ClusteringField field="field_1" compareFunction="absDiff"/>
 <ClusteringField field="field_2" compareFunction="absDiff"/>
 <ClusteringField field="field_3" compareFunction="absDiff"/>
 <ClusteringField field="field_4" compareFunction="absDiff"/>
 <ClusteringField field="field_5" compareFunction="absDiff"/>
 <ClusteringField field="field_6" compareFunction="absDiff"/>
 <ClusteringField field="field_7" compareFunction="absDiff"/>
 <ClusteringField field="field_8" compareFunction="absDiff"/>
 <ClusteringField field="field_9" compareFunction="absDiff"/>
 <ClusteringField field="field_10" compareFunction="absDiff"/>
 <ClusteringField field="field_11" compareFunction="absDiff"/>
 <Cluster name="cluster_0">
 <Array n="12" type="real">0.5788327370461465 0.0338725581836942 0.020011301098778453 0.1666257715575919 0.0056309725158454 0.40349633343290753 0.9675213942134802 0.06601698929968569 0.2180910512034508 0.04238828475464813 0.714994401855638 0.38312153624625206</Array>
 </Cluster>
 <Cluster name="cluster_1">
 <Array n="12" type="real">0.18039444291409718 0.011903512685698184 0.00538682352254432 0.052890252892119106 0.001927570269812699 0.3694765387729449 1.0 0.022183009281431215 0.0741005521280797 0.014625924011909038 0.22995980340258093 0.12500312447932746</Array>
 </Cluster>
 </ClusteringModel>
</PMML>